<img src="advenica-logo.svg" width="150" align="right">

# Intrusion detection Part 1 - MySQL logs

The scenario:

A hacker is trying to gain access to a system through the following steps:

1. Open Port Scan:
    - A simple open port scan of the system reveals that the MySQL port (3306) is open.
2.  Gathering Information with `nmap`:
    - Using `nmap` and its handy `--script` flag gives some useful information about the MySQL service running on the system.
3. Brute-Forcing MySQL Credentials:
    - Suspecting that the MySQL database might contain valuable data, Metasploit's `mysql_login` auxiliary module is used to brute-force the server's credentials through a list of common users and potential passwords. 
    - The superuser (root) of the database server had a weak password.
4. Accessing the MySQL Database:
    - Upon logging into the MySQL database, there is information related to a WordPress site, including a table with hashed passwords.
    - These weakly hashed passwords are easily cracked, revealing plaintext passwords to the site.

_(Continued in part 2)_

# Investigating the incident

We start our investigation with examining the logs from the mysql database to see if we can find any evidence of an intrusion attempt there.

In [ ]:
# Import some useful python modules

!pip -q install  pandas seaborn matplotlib scikit-learn logparser3 drain3 numpy

## What the modules do and links to documentation 

- [pandas](https://pandas.pydata.org/docs/) is a data analysis and manipulation tool
- [seaborn](https://seaborn.pydata.org/tutorial.html) is a statistical data visualization tool
- [matplotlib](https://matplotlib.org/stable/users/index) is a library for creating visualizations
- [scikit-learn](https://scikit-learn.org/stable/user_guide.html) is a machine learning and predictive data analysis tool
- [logparser3](https://github.com/logpai/logparser) is a machine learning toolkit for automated log parsing
- [drain3](https://github.com/logpai/Drain3) is an online log template miner
- [numpy](https://numpy.org/doc/stable/) is a package for scientific computing used by the above tools 

## Configure pandas display options

In order to keep output readable we change a couple of default settings

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)          # Show all columns
pd.set_option('display.max_colwidth', 1000)         # Set a large max column width
pd.set_option('display.colheader_justify', 'left')  # Justify column headers to the left
pd.set_option('display.expand_frame_repr', False)   # Prevent line breaks

# Preprocessing

Preprocessing of the log file to make it more suitable for our log analyzer (drain).

This is a messy step, because it addresses the gap, or impedence mismatch if you want, between the log file format that you have to work with and the log file format that you _wish_ that you had. This step is always going to be present in a real application, but unfortunately will depend on the circumstances, so consider this an example of what it _could_ look like. 

- We parse the SQL statements & the table operated on since these can both be potential features of interest
- Timestamps are expected to look like `2024-07-08T10:27:53.653910Z` (so in UTC)
- Log entries are expected look like this: 
    - `2024-07-10T05:25:05.058758Z49509 Query  SELECT * FROM wp_posts WHERE ID = 47 LIMIT 1`
    - *Note 1*: the number after Z in the timestamp is actually a mysql ID number for the transaction.
    - *Note 2*: The only command with a space is `Init DB` - in order to make it more manageable, we replace it with `InitDB`.
- Log entries can contain newlines so we use the fact that a timestamp marks start of the next entry.
    - We use split to remove unnecessary whitespace (tabs, newlines, spaces and multiples thereof)
- After preprocessing, the log will only contain single line entries on the form
    - `<timestamp> <id> <command> <sql action (if any)> <table (if any>)> <and anything else>`
    - `2024-07-08T10:27:53.654594Z 225 Query SELECT wp_posts SELECT * FROM wp_posts WHERE ID = 21 LIMIT 1`
- Finally, we store the log entries in a pandas [dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame) with columns `timestamp`, `id`, `command`, `action`, `table`, and `argument`.
    - If any optional part (`action`, `table`, and `argument`) is missing, a `-` is used as a placeholder.

In [ ]:
import os
import os.path
import gzip
import re

import pandas as pd

timestamp_re = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{6}Z)')
logline_re = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{6}Z) *(\d+) (Init DB|\S+)(?: ?(.*))$')

# Regex parsing of SQL statements to get the action
update_re = re.compile(r'UPDATE (?:`)(\S*)(?:`) .*')
select_w_table_re = re.compile(r'SELECT\s+.*\s*FROM\s*(\S+).*')
select_wo_table_re = re.compile(r'SELECT\s+.*')
delete_re = re.compile(r'DELETE\s+FROM\s*`(\S+)`.*')
show_re = re.compile(r'SHOW.*`(\S+)`')
insert_re = re.compile(r'INSERT INTO\s+.*`(\S+)`.*')
set_re = re.compile(r'SET .*')
flush_re = re.compile(r'FLUSH .*')

def get_sql_action(entry):
    if select_w_table_re.match(entry):
         return "SELECT", select_w_table_re.match(entry).group(1)
    if select_wo_table_re.match(entry):
         return "SELECT", "-"
    if delete_re.match(entry):
         return "DELETE", delete_re.match(entry).group(1)
    if insert_re.match(entry):
         return "INSERT", insert_re.match(entry).group(1)
    if update_re.match(entry):
         return "UPDATE", update_re.match(entry).group(1)
    if set_re.match(entry):
         return "SET", "-"
    if show_re.match(entry):
         return "SHOW", show_re.match(entry).group(1)
    if flush_re.match(entry):
         return "FLUSH", "-"
    return '-', '-'

def parse_log(file_path):
    log_entries = []
    ofile, _ = os.path.splitext(os.path.basename(file_path))
    if not os.path.exists("preproc_data"):
        os.mkdir("preproc_data")
    with gzip.open(file_path, 'rt', encoding="utf8") as file, \
         open(f"preproc_data/preproc-{ofile}", "wt", encoding="utf8") as of:
        full_entry = ""
        for line in file:
            if line.startswith("/usr/sbin/mysqld") or \
                line.startswith("Tcp port:") or \
                line.startswith("Time    "):
                   print(f"Skipping: {line.strip()}")
                   continue
            if timestamp_re.match(line) and len(full_entry) > 0:
               # Split removes unnecessary whitespace
               full_entry = " ".join(full_entry.split())
               match = logline_re.match(full_entry)
               if match:
                    new_entry = list(match.groups())
                    arguments = new_entry.pop()
                    if new_entry[2] == 'Query':
                         new_entry.extend(get_sql_action(arguments))
                    elif new_entry[2] == 'Init DB':
                         new_entry[2] = 'InitDB' 
                         new_entry.extend(['-', '-'])
                    else:
                         new_entry.extend(['-', '-'])
                    new_entry.append(arguments if arguments else '-')
                    log_entries.append(new_entry)
                    of.write(" ".join([ s for s in new_entry if s]) + "\n")
               else:
                    print(f"skipping '{full_entry}'")
               full_entry = line.strip()
            else:
               full_entry += line
    return pd.DataFrame(log_entries, columns=['timestamp', 'id', 'command', 'action', 'table', 'argument'])

df = parse_log('raw_data/general.log.gz')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['id'] = pd.to_numeric(df['id'], downcast='unsigned')

In [ ]:
df.dtypes

# Anomalies

For the training data, we happen to know which log id's are part of an anomaly.

We will make use of this later.

In [ ]:
# label anomalies
login_attack = [2523, 2611] + list(range(2630, 2700))
successful_login  = [2541, 2700]

# Some data visualisations

Just a couple of plots showing what the data looks like.

In [ ]:
import seaborn
import matplotlib.pyplot as plt

# Visualizations
seaborn.countplot(data=df, x='action')
plt.title('Distribution of SQL Actions')
plt.show()

In [ ]:
# Visualizations
fig = seaborn.countplot(data=df, x='table')
fig.tick_params("x", labelrotation=90)
plt.title('Distribution of SQL table used')
plt.show()

In [ ]:
df.dtypes

In [ ]:
# FIXME: Not idempotent since set_index(, inplace=True) destroys original format
df.set_index('timestamp', inplace=True)
df['hour'] = df.index.hour
hourly_patterns = df.groupby(['action', 'hour']).size().unstack().T
hourly_patterns.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('SQL Actions by Hour of Day')
plt.show()

In [ ]:
df.dtypes

In [ ]:
df['hour'] = df.index.hour
hourly_patterns = df.groupby(['table', 'hour']).size().unstack().T
hourly_patterns.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('SQL table by Hour of Day')
plt.show()

In [ ]:
frequent_patterns = df.groupby(['action', 'table']).size().reset_index(name='counts')
frequent_patterns.sort_values(by='counts', ascending=False, inplace=True)
print(frequent_patterns.head(10))

In [ ]:
thresholds = df.groupby('action')['id'].describe(percentiles=[.01, .99])
print(thresholds)

thresholds = df.groupby('table')['id'].describe(percentiles=[.01, .99])
print(thresholds)

In [ ]:
import contextlib

with contextlib.redirect_stdout(None):
    print("will not print")

print("this will print")

# Drain & log parsing

Next, we use drain to parse the (pre-processed) logs.

In [ ]:
from logparser.Drain import LogParser

input_dir = 'preproc_data/' # The input directory of log file
output_dir = 'result/'  # The output directory of parsing results
log_file = 'preproc-general.log'
# time id command argument
log_format = '<Timestamp>Z <Id> <Command> <Action> <Table> <Content>' # Define log format to split message fields
# Regular expression list for optional preprocessing (default: [])
regex = [
    r'([\da-fA-F]{8,})', # HEX numbers
    r'(\d{5,})', # 'large' integers
    r'(\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})' # IP numbers
]
st = 0.5  # Similarity threshold
depth = 4  # Depth of all leaf nodes

parser = LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)

import contextlib

with contextlib.redirect_stdout(None):
    parser.parse(log_file)

# Parsed log data

Now the data is parsed and has a bit of structure, so we can load it into a dataframe and work with that.

In [ ]:
# Load structured data
dtypes = {
    'LineId': 'int64',
    'Timestamp': 'str',
    'Id': 'int64',
    'Command': 'str',
    'Action': 'str',
    'Table': 'str',
    'Content': 'str',
    'EventId': 'str',
    'EventTemplate': 'str',
    'ParameterList': 'str'
}
df = pd.read_csv('result/preproc-general.log_structured.csv', dtype=dtypes)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
templates = pd.read_csv('result/preproc-general.log_templates.csv')

# Supervised vs Unsupervised

Since we have the labels of the data, we can add those classifiers to the structured data.

The classes are:
  - `normal` for lines that are not part of an attack.
  - `login_attack` for lines that are part of an attempt to brute force database logins.
  - `get_user_table` for lines that are part of an attack which downloads the user table.


In [ ]:
def mark_anomalies(row):
    if row['Id'] in login_attack:
        return 'login_attack'
    if row['Id'] in successful_login:
        return 'get_user_table'
    return 'normal'

df['Classification'] = df.apply(mark_anomalies, axis=1)
df['Anomaly'] = df.apply(lambda row: -1 if row['Classification'] != 'normal' else 1, axis=1)
df

# Encoding

When training a model, it's mostly numbers that matter, so we encode the features we think are relevant.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['EncodedEventId'] = label_encoder.fit_transform(df['EventId'])
df['EncodedCommand'] = label_encoder.fit_transform(df['Command'])
df['EncodedAction'] = label_encoder.fit_transform(df['Action'])
df['EncodedTable'] = label_encoder.fit_transform(df['Table'])


In [ ]:
df

# Isolation forest, unsupervised

Let's take the preprocessed, parsed & encoded data for a spin with isolation forest.

This is not terrible. Right out of the box, it is really good at identifying normal logs and at times not terrible at identifying anomalies. 

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

ldf = df[['EncodedEventId', 'EncodedCommand', 'EncodedAction', 'EncodedTable']]
features = df[['EncodedEventId', 'EncodedCommand', 'EncodedAction', 'EncodedTable']]

# Train Isolation Forest model
model = IsolationForest(contamination=0.01)
ldf['PredictedAnomaly'] = model.fit_predict(features)

# Generate the confusion matrix
cm = confusion_matrix(df['Anomaly'], ldf['PredictedAnomaly'], labels=[1, -1])

# Calculate performance metrics
report = classification_report(df['Anomaly'], ldf['PredictedAnomaly'], labels=[1, -1], target_names=['Normal', 'Anomaly'])
precision = precision_score(df['Anomaly'], ldf['PredictedAnomaly'], pos_label=-1)
recall = recall_score(df['Anomaly'], ldf['PredictedAnomaly'], pos_label=-1)
f1 = f1_score(df['Anomaly'], ldf['PredictedAnomaly'], pos_label=-1)

print("Classification Report:")
print(report)

# Normalize the confusion matrix by row (i.e by the number of samples in each actual class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot the confusion matrix as a heatmap with counts and percentages
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Normal', 'Predicted Anomaly'], yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix with Counts')
plt.show()

plt.figure(figsize=(10, 7))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', xticklabels=['Predicted Normal', 'Predicted Anomaly'], yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix with Percentages')
plt.show()



# Random forest, supervised

Next, we make use of the labels and try out random forest.

This approach is quite successful since there is very little overlap between anomalous and non-anomalous events.

In [ ]:
# label anomalies
login_attack = [2523, 2611] + list(range(2630, 2700))
successful_login  = [ 2541, 2700]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


X = df[['EncodedEventId', 'EncodedCommand', 'EncodedAction', 'EncodedTable']]
y = df['Anomaly']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[1, -1])

# Print the classification report
report = classification_report(y_test, y_pred, target_names=['Normal', 'Anomaly'])
print("Classification Report:")
print(report)

# Plot the confusion matrix as a heatmap with counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Normal', 'Predicted Anomaly'], yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix with Counts')
plt.show()

# Normalize the confusion matrix by row (i.e., by the number of samples in each actual class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot the confusion matrix as a heatmap with percentages
plt.figure(figsize=(10, 7))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', xticklabels=['Predicted Normal', 'Predicted Anomaly'], yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix with Percentages')
plt.show()


# Explanation

The supervised approach is surprisingly successful without any extra bells and whistles. The reason for this is (probably) that there is very little overlap between the anomalous and non-anomalous log entries as seen below. Only three events occur as both anomalies and non-anomalies; the rest are only found in one of them.

In [ ]:
anomalies = list(set(df[df['Anomaly'] == -1]['EventId']))
normies = list(set(df[df['Anomaly'] == 1]['EventId']))

mixed = [ x for x in normies if x in anomalies]
print(len(mixed))